# `CLOUD.jl` - 3D linear advection example

In [8]:
using CLOUD, OrdinaryDiffEq

## Define the physical problem

In [9]:
θ = π/4 # wave angle
a = sqrt(2) # wave speed
A = 1.0  # amplitude
L = 1.0  # domain length
k = (2*π/L, 2*π/L, 2*π/L)  # wave number
T = 1.0/(a*max(abs(cos(θ)),abs(sin(θ)))) # end time for one period

conservation_law = LinearAdvectionEquation((a*cos(θ),a*sin(θ),0.0))
initial_data = InitialDataCosine(A,k)
exact_solution = ExactSolution(conservation_law,initial_data);

## Set discretization parameters (standard DG)

In [10]:
M = 4
p = 4
p_map = 2
form = WeakConservationForm(mapping_form=StandardMapping(), 
    inviscid_numerical_flux=LaxFriedrichsNumericalFlux(1.0))
strategy = PhysicalOperator()

ode_algorithm = CarpenterKennedy2N54()
dt = 0.25*(L/M)/(a*(2*p+1));
write_interval = floor(Int, T/(dt*50));

## Set up a `ModalMulti` scheme of degree `p` with `M` edges in each direction

In [11]:
reference_approximation = ReferenceApproximation(
    ModalMulti(p), Tet(), mapping_degree=p_map)

mesh = warp_mesh(uniform_periodic_mesh(
    reference_approximation.reference_element, 
    ((0.0,L),(0.0,L),(0.0,L)), (M,M,M)), 
    reference_approximation.reference_element, 0.1) 

spatial_discretization = SpatialDiscretization(mesh, 
    reference_approximation)

results_path = save_project(conservation_law,
     spatial_discretization, initial_data, form, (0.0, T), strategy,
     "results/advection_3d/", overwrite=true, clear=true);

## Run the solver

In [12]:
ode_problem_1 = semidiscretize(conservation_law,
    spatial_discretization,
    initial_data, 
    form,
    (0.0, T),
    strategy);

save_solution(ode_problem_1.u0, 0.0, results_path, 0)

CLOUD_reset_timer!()
sol = solve(ode_problem_1, ode_algorithm, adaptive=false, dt=dt,
    save_everystep=false, 
    callback=save_callback(results_path, write_interval))
CLOUD_print_timer()

save_solution(last(sol.u), last(sol.t), results_path, "final")

 ────────────────────────────────────────────────────────────────────────────────
            Thread 1                    Time                    Allocations      
                               ───────────────────────   ────────────────────────
       Tot / % measured:            19.4s /  95.9%           80.8GiB /  99.7%    

 Section               ncalls     time    %tot     avg     alloc    %tot      avg
 ────────────────────────────────────────────────────────────────────────────────
 local residual          392k    17.7s   94.8%  45.0μs   79.9GiB   99.2%   214KiB
   gather ext state      392k    9.60s   51.5%  24.5μs   72.3GiB   89.8%   193KiB
   apply operators       392k    3.94s   21.1%  10.0μs   2.63GiB    3.3%  7.03KiB
     volume terms        392k    2.23s   11.9%  5.68μs   1.67GiB    2.1%  4.45KiB
     facet terms         392k    868ms    4.7%  2.21μs    604MiB    0.7%  1.58KiB
   eval num flux         392k    1.18s    6.4%  3.02μs   2.97GiB    3.7%  7.95KiB
   eval nodal s

## Calculate the integral L<sup>2</sup> error norm and conservation/energy error

In [13]:
error_analysis = ErrorAnalysis(results_path, conservation_law, 
    spatial_discretization)
conservation_analysis = PrimaryConservationAnalysis(results_path, 
    conservation_law, spatial_discretization)
energy_analysis = EnergyConservationAnalysis(results_path, 
    conservation_law, spatial_discretization)

println("L2 error:")
println(analyze(error_analysis, last(sol.u), exact_solution, T)...)
println("Conservation (initial/final/diff):")
println(analyze(conservation_analysis)...)
println("Energy (initial/final/diff):")
println(analyze(energy_analysis)...)

L2 error:
0.0044161318102064655
Conservation (initial/final/diff):
[-4.743384504624082e-19][1.0640089070229619e-16][1.068752291527586e-16]
Energy (initial/final/diff):
[0.062499251276950024][0.062375030258773674][-0.00012422101817635006]


## Write output to VTK file

In [14]:
for i in eachindex(sol.u)
    postprocess_vtk(spatial_discretization, 
        string(results_path,"vis_", lpad(i,3,"0"), ".vtu"),
            sol.u[i],p_map=p_map,p_vis=p)
end